# Análise dos produtos comercializados

## Criar directorias de apoio 

In [2]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [3]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = APIKEYHERE
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


## Inicializar a API e transferir os códigos usados

In [4]:
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    APIKEY = config['comtrade']['key']

comtrade.init(APIKEY)

## China, categorias de produtos mais importantes nas trocas com os PLP

Obtem os totais agregados com nível 2 de código HS e lista os primeiros.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx


### TODO

- [ ] Permitir escolher os reporter e partners
- [ ] compatibilizar com o formato de report da AICEP https://myaicep.portugalexporta.com/mercados-internacionais/cn/china?setorProduto=-1
- [ ] há problemas de duplicação de linhas nas listagem de detalhes de commodities, provavlmente por custom code, ou mot code.
  

In [5]:
import comtrade
import pandas as pd


rank_filter = 5  # número de importações mais relevantes

year_start = 2020
year_end = 2023
years = comtrade.year_range(year_start,year_end)
reporterCode = comtrade.m49_china
partnerCode = comtrade.m49_plp_list
flowCode = 'M,X'
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
            'flowCode','primaryValue']



pco_top5_sorted = comtrade.top_commodities(reporterCode, partnerCode,  years, flowCode,rank_filter, pco_cols)

# save to Excel
# Prepare file name for Excel output
filename_note=f"{year_start}-{year_end}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./reports/china_plp_top5_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)

# Save data
sheet_name = "comtrade"
pco_top5_sorted.to_excel(excel_file, sheet_name=sheet_name, index=True, startrow=2)
# Adjust column width
comtrade.excel_col_autowidth(pco_top5_sorted, excel_file,sheet_name)

# format column primaryValue as currency
comtrade.excel_format_currency(pco_top5_sorted, excel_file,sheet_name, columns=['primaryValue'],width=20)
    
# Write title in first row TBD
excel_file_title = f"Trocas Comerciais China-PLP, top {rank_filter} produtos (M=Importações, X=Exportações), {years}, valores USD"
excel_file.sheets[sheet_name].write(0, 0, excel_file_title)
print("Guardado em:",excel_file_name)

excel_file.close()
# show
pd.options.display.max_colwidth=100
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 100
pco_top5_sorted.head(50)

Guardado em: ./reports/china_plp_top5_2020-2023_M_X.xlsx


cmdCode  \
reporterDesc partnerDesc flowCode refYear rank           
China        Angola      M        2020    1         27   
                                          2         25   
                                          3         71   
                                          4         74   
                                          5         26   
                                  2021    1         27   
                                          2         25   
                                          3         26   
                                          4         74   
                                          5         76   
                         X        2020    1         85   
                                          2         84   
                                          3         64   
                                          4         72   
                                          5         39   
                                  2021    1         85   
                                          2         84   
                                          3         87   
                                          4         39   
                                          5         63   
             Brazil      M        2020    1         26   
                                          2         12   
                                          3         27   
                                          4         02   
                                          5         47   
                                  2021    1         26   
                                          2         12   
                                          3         27   
                                          4         02   
                                          5         47   
                         X        2020    1         85   
                                          2         84   
                                          3         29   
                                          4         90   
                                          5         87   
                                  2021    1         85   
                                          2         84   
                                          3         29   
                                          4         72   
                                          5         90   
             Cape Verde  M        2020    1         74   
                                          2         84   
                                          3         85   
                                          4         99   
                                          5         63   
                                  2021    1         74   
                                          2         26   
                                          3         62   
                                          4         49   
                                          5         99   

                                                                                                                                            cmdDesc  \
reporterDesc partnerDesc flowCode refYear rank                                                                                                        
China        Angola      M        2020    1     Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral w...   
                                          2                                     Salt; sulphur; earths, stone; plastering materials, lime and cement   
                                          3     Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with prec...   
                                          4                                                                             Copper and articles thereof   
                                          5                                                                                 

## Análise de variações nos produtos mais importantes

A célula seguinte produz uma lista das variações mais importantes na evolução das 
trocas comerciais entre a China e os Países de Língua Portuguesa

Por cada combinação de países e de fluxo (M=importações pela China, X=exportações da China)
por exemplo:

        China Brazil M

        1993   72,26
                -> 26   141,782,165.00 Ores, slag and ash
                -> 72   540,603,898.00 Iron and steel
        1994 ! 15,72
                -> 15   463,690,083.00 Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes
        1995 ! 15,26
        1996 ! 15,23
                -> 23   270,712,928.00 Food industries, residues and wastes thereof; prepared animal fodder
        1997 ≈ 23,15
        1998 ! 26,23
        1999 ! 26,12
                -> 12   171,963,034.00 Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder
        2000 ≈ 12,26
        2001 ≈ 26,12
        2002 ≈ 12,26
        2003 =
        2004 ≈ 26,12

Cada ano é seguido de um sinal que indica a mudança ocorrida em relação ao ano anterior:
* ! : mudança num dos produtos principais
* = : mesmos produtos do ano anterior, pela mesma ordem de importância
* ≈ : mesmos produtos do ano anterior, mas importância diferente


Cada vez que um novo produto aparece na série é fornecida a descrição do código respectivo e o valor envolvido.

Esta listagem pode ser produzida tendo em conta um número variáveis de produtos principais
normalmente os dois principais (ver variável `consider_first`)

In [6]:
consider_first = 2
for reporter in pco_top5_sorted.index.get_level_values(0).unique():
    for partner in pco_top5_sorted.loc[(reporter,)].index.get_level_values(0).unique(): 
        for flow in pco_top5_sorted.loc[((reporter,partner,))].index.get_level_values(0).unique():
            print(reporter,partner, flow)
            previous_pattern = []
            all_commodities = set() # all commodities for this partner for this flow
            all_commodities_desc = set() # all commodities for this partner for this flow descriptions
            previous_years_values = dict()
            for year in pco_top5_sorted.loc[(reporter,partner,flow,)].index.get_level_values(0).unique():
                print(year, end=' ')
                top5_codes = pco_top5_sorted.loc[(reporter,partner,flow,year)]['cmdCode'].astype(str).values.flatten().tolist()[0:consider_first]
                top5_codes_desc = pco_top5_sorted.loc[(reporter,partner,flow,year)]['cmdDesc'].astype(str).values.flatten().tolist()[0:consider_first]
                top5_primary_value = pco_top5_sorted.loc[(reporter,partner,flow,year)]['primaryValue'].values.flatten().tolist()[0:consider_first]
                top5_table = {code: {"desc":desc, "value":value, "year": year} for code,desc,value in zip(top5_codes,top5_codes_desc,top5_primary_value)}
                codes_desc = {f"{code} {desc}" for code,desc in zip(top5_codes,top5_codes_desc)}
                # make set of tuples top5_codes and top5_codes_desc
                new_commodities = set(top5_codes).difference(all_commodities)
                all_commodities = all_commodities.union(set(top5_codes))
                all_commodities_desc = all_commodities_desc.union(codes_desc)
                pattern = ','.join(top5_codes)
                if len(previous_pattern) == 0:
                    previous_pattern = top5_codes
                    # update previous_years_values with top5_table
                    previous_years_values.update(top5_table)
                    print(' ', pattern, end=' | ')
                    for cmd in top5_codes:
                        print(f"{top5_table[cmd]['value']:,.2f} ({top5_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                    for cmd in new_commodities:
                        print( " "*13+"  ->", cmd,top5_table[cmd]['desc'])
                elif set(top5_codes) == set(previous_pattern):
                    if top5_codes == previous_pattern:
                        print('=', pattern, end=' | ')
                    else:
                        print('≈', pattern, end=' | ')
                    for cmd in top5_codes:
                        print(f"{top5_table[cmd]['value']:,.2f} ({top5_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                else:
                    print('!', pattern, end=' | ')
                    for cmd in top5_codes:
                        print(f"{top5_table[cmd]['value']:,.2f} ", end=' ')
                        if cmd in previous_years_values.keys():
                            print(f"({top5_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                        else:
                            print(f"(new:{cmd})", end=' | ')
                    print()
                    if len(new_commodities) > 0:
                        for cmd in new_commodities:
                            print( " "*13+"  ->", cmd, top5_table[cmd]['desc'])
                       
                previous_pattern = top5_codes
                previous_years_values.update(top5_table)
            for cmd in sorted(all_commodities_desc):
                print( "    ",cmd)
            print(80*"-")



China Angola M
2020   27,25 | 14,686,950,196.00 (100%) | 29,428,961.00 (100%) | 
               -> 27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
               -> 25 Salt; sulphur; earths, stone; plastering materials, lime and cement
2021 = 27,25 | 20,823,873,963.00 (142%) | 30,420,529.00 (103%) | 
     25 Salt; sulphur; earths, stone; plastering materials, lime and cement
     27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
--------------------------------------------------------------------------------
China Angola X
2020   85,84 | 203,996,443.00 (100%) | 137,859,846.00 (100%) | 
               -> 85 Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles
               -> 84 Machinery and mechanical appliances, boilers, nuclear reactors; parts there

## Detalhe do tipo de produtos comercializados entre os PLP e a China

Em dois passos: 
* obtém as categorias de nível 2 mais importantes de cada país
* pesquisa todos as subcategorias de cada um das categorias

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [7]:

import ipywidgets as widgets
from IPython.display import display
import comtrade

rank_filter = 5  # número de importações mais relevantes
years = "2021"
partnerCode = comtrade.m49_angola # 
flowCode="M"

# select year with widget from list range(2000,2022)

years = widgets.SelectMultiple(
    options=range(2000,2022),
    value=[2021],
    description='Ano:',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
reporterCodeWidget = widgets.Dropdown(
    options=[("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE ,
    # value=[("Angola",m49_angola)],
    description='Fonte (reporter):',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
partnerCodeWidget = widgets.Dropdown(
    options=[("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE ,
    # value=[("Angola",m49_angola)],
    description='Parceiro:',
    disabled=False,
)

# select flowCode widget from list M=Importações, X=Exportações
flowCodeWidget = widgets.Dropdown(
    options=[("Importações", "M"), ("Exportações", "X")],
    value="M",
    description='Fluxo:',
    disabled=False, 
)



### Escolher ano, parceiro e fluxo (import/export) para análise 

Pode escolher-se mais que um ano.

In [11]:

display(years)
display(reporterCodeWidget)
display(partnerCodeWidget)
display(flowCodeWidget)


SelectMultiple(description='Ano:', index=(17, 18, 19, 20, 21), options=(2000, 2001, 2002, 2003, 2004, 2005, 20…

Dropdown(description='Fonte (reporter):', options=(('China', 156), ('Macau', 446), ('Hong Kong', 344), ('Angol…

Dropdown(description='Parceiro:', index=3, options=(('China', 156), ('Macau', 446), ('Hong Kong', 344), ('Ango…

Dropdown(description='Fluxo:', options=(('Importações', 'M'), ('Exportações', 'X')), value='M')

Depois de escolher ano, país e fluxo executar a célula seguinte.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [17]:
import comtrade
from comtrade import COUNTRY_CODES, HS_CODES, HS_CODES_DF, HS_CODES_L2_DF


yearsList = ",".join(list(map(str,years.value)))
partnerCode = partnerCodeWidget.value
reporterCode = reporterCodeWidget.value


# decode country
partner_name = comtrade.COUNTRY_CODES.get(partnerCode)
reporter_name = comtrade.COUNTRY_CODES.get(reporterCode)

flow = flowCodeWidget.value
print(reporter_name,yearsList,flow,partner_name,)

pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']

pco_cols_detail = ['reporterDesc','partnerDesc','refYear','cmdCodeAG2','cmdCode','cmdDesc',
            'flowCode','primaryValue', 'isAggregate']

# first we collect the top commodity level 2 codes            
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     cmdCode="AG2",
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     period=yearsList,
                     timeout=30,
                     echo_url=True
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
# get the countries
countries = pco_top5.partnerDesc.unique()
country_cmd_top5_codes = dict()

# get the detailed commodity codes for the top of each country
for country in countries:
    l2_codes = pco_top5[pco_top5.partnerDesc == country]['cmdCode'].unique()
    print(country,l2_codes)
    hs_details = []
    for l2_code in l2_codes:
        l2_sub_codes = list(HS_CODES_DF[HS_CODES_DF.hscode.str.startswith(l2_code)]['hscode'])
        hs_details = hs_details + l2_sub_codes
    # print(hs_details)
    country_cmd_top5_codes[country] = hs_details.copy()

# now we fetch the detail
countryDesc = COUNTRY_CODES[partnerCode]
country_cmd_codes = ",".join(country_cmd_top5_codes[countryDesc])
print(country_cmd_codes)

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     #cmdCode=country_cmd_codes, # sometimes the detail commodity codes generates errors
                     cmdCode=None, # This gives all the commodities for the country and year 
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     partner2Code=0,
                     period=yearsList,
                     echo_url=True,
                     timeout=60
                     )
                     
df['cmdCodeAG2'] = df.cmdCode.str[0:2]
# filter the detail commodity codes by the top 5
df = df[df['cmdCodeAG2'].isin(country_cmd_top5_codes[countryDesc])]

China 2018 M Angola
https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=2018&partnerCode=24&partner2Code=0&cmdCode=AG2&flowCode=M&customsCode=C00&subscription-key=HIDDEN
Angola ['27' '44' '25' '71' '26']
27,2701,270111,270112,270119,270120,2702,270210,270220,2703,270300,2704,270400,2705,270500,2706,270600,2707,270710,270720,270730,270740,270750,270791,270799,2708,270810,270820,2709,270900,2710,271012,271019,271020,271091,271099,2711,271111,271112,271113,271114,271119,271121,271129,2712,271210,271220,271290,2713,271311,271312,271320,271390,2714,271410,271490,2715,271500,2716,271600,44,4401,440111,440112,440121,440122,440131,440132,440139,440141,440149,4402,440210,440220,440290,4403,440311,440312,440321,440322,440323,440324,440325,440326,440341,440342,440349,440391,440393,440394,440395,440396,440397,440398,440399,4404,440410,440420,4405,440500,4406,440611,440612,440691,440692,4407,440711,440712,440713,440714,440719,440721,440722,440723,440725,440726,440727,440728,44072

Temos que identificar as linhas com valor agregado porque
os dados não tem essa coluna correcta.

Na lista China-Angola importações a flag isAggregate parece não 
estar correct até 2017 inclusive. 2018 já está bem


In [18]:
#set the number of output lines to 200
pd.set_option('display.max_rows', 200)

df.sort_values(['reporterDesc','partnerDesc','refYear','flowCode','cmdCode'], inplace=True)
df[['reporterDesc','partnerDesc','refYear','flowCode','cmdCodeAG2','cmdCode','isAggregate']].head(200)

,reporterDesc,partnerDesc,refYear,flowCode,cmdCodeAG2,cmdCode,isAggregate
12,China,Angola,2018,M,25,25,True
13,China,Angola,2018,M,25,2506,True
14,China,Angola,2018,M,25,250610,False
15,China,Angola,2018,M,25,250620,False
16,China,Angola,2018,M,25,2515,True
17,China,Angola,2018,M,25,251511,False
18,China,Angola,2018,M,25,251512,False
19,China,Angola,2018,M,25,2516,True
20,China,Angola,2018,M,25,251611,False
21,China,Angola,2018,M,25,251612,False


In [22]:
import comtrade
# st max lines to 200
pd.set_option('display.max_rows', 200)

# check which lines are aggregates
df = comtrade.checkAggregateValues(df,'cmdCode','isAggregate2')
df.sort_values(['reporterDesc','partnerDesc','refYear','flowCode','cmdCode'], inplace=True)
df[['reporterDesc','partnerDesc','refYear','flowCode','cmdCodeAG2','cmdCode','isAggregate','isAggregate2']].head(200)


,reporterDesc,partnerDesc,refYear,flowCode,cmdCodeAG2,cmdCode,isAggregate,isAggregate2
12,China,Angola,2018,M,25,25,True,True
13,China,Angola,2018,M,25,2506,True,True
14,China,Angola,2018,M,25,250610,False,NaN
15,China,Angola,2018,M,25,250620,False,NaN
16,China,Angola,2018,M,25,2515,True,True
17,China,Angola,2018,M,25,251511,False,NaN
18,China,Angola,2018,M,25,251512,False,NaN
19,China,Angola,2018,M,25,2516,True,True
20,China,Angola,2018,M,25,251611,False,NaN
21,China,Angola,2018,M,25,251612,False,NaN


In [ ]:

# list unique values of MultiIndex
lastCode = "---"
lastIndex = 0
for row in df.iterrows():
    currentCode = row[1]['cmdCode']
    if currentCode != lastCode and currentCode.startswith(lastCode):
        # print(f">>>> Last code {lastCode} index {lastIndex} is parent of {currentCode}")
        df.loc[lastIndex,'isAggregate'] = True
    # print(df.loc[row[0]][['cmdCode','cmdDesc']])
    lastCode = currentCode
    lastIndex = row[0]
df['valueForTotal'] = df['primaryValue']
df.loc[df['isAggregate'] == True,'valueForTotal'] = 0
df['subtotalAG2'] = df.groupby(['partnerCode','refYear','flowCode','cmdCodeAG2'])["valueForTotal"].transform('sum')
df['percentAG2'] = df['primaryValue'] / df['subtotalAG2'] * 100

In [20]:
import comtrade

excel_file_name = f"./reports/product_detail_{reporter_name}_{yearsList.replace(',','-')}_{flow}_{partner_name}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name, engine='xlsxwriter')

df2 = df[pco_cols_detail+['percentAG2']].sort_values('primaryValue',ascending=False)
df2.to_excel(excel_file, sheet_name='detail', index=True, startrow=2)

comtrade.excel_col_autowidth(df2, excel_file, 'detail')
comtrade.excel_format_currency(df2, excel_file,'detail', columns=['primaryValue'],width=20)
excel_file.close()
print(f"Excel file {excel_file_name} created")

pd.options.display.max_rows = 500
# df.sort_values(['partnerDesc','refYear','flowCode','subtotalAG2','cmdCodeAG2','primaryValue'],ascending=[True,True,True,False,True,False])[pco_cols_detail].head(500)
df[df['isAggregate']==False].sort_values(['reporterDesc','partnerDesc','refYear','subtotalAG2','cmdCode'],ascending=[True,True,True,False,True])[pco_cols_detail+['percentAG2']].head(500)


Excel file ./reports/product_detail_China_2018_M_Angola.xlsx created


,reporterDesc,partnerDesc,refYear,cmdCodeAG2,cmdCode,cmdDesc,flowCode,primaryValue,isAggregate,percentAG2
34,China,Angola,2018,27,270900,"Oils; petroleum oils and oils obtained from bituminous minerals, crude",M,"25,037,434,241.00",False,97.21
36,China,Angola,2018,27,271111,"Petroleum gases and other gaseous hydrocarbons; liquefied, natural gas",M,"297,983,624.00",False,1.16
37,China,Angola,2018,27,271112,"Petroleum gases and other gaseous hydrocarbons; liquefied, propane",M,"353,675,355.00",False,1.37
38,China,Angola,2018,27,271113,"Petroleum gases and other gaseous hydrocarbons; liquefied, butanes",M,"66,208,694.00",False,0.26
57,China,Angola,2018,44,440122,"Wood; for fuel, in chips or particles, non-coniferous, whether or not agglomerated",M,"5,512.00",False,0.02
59,China,Angola,2018,44,440349,"Wood, tropical; other than dark red meranti, light red meranti meranti bakau and teak, in the ro...",M,"25,191,487.00",False,81.64
60,China,Angola,2018,44,440398,"Wood; of eucalyptus (Eucalyptus spp.), in the rough, whether or not stripped of bark or sapwood,...",M,"93,149.00",False,0.30
61,China,Angola,2018,44,440399,"Wood; in the rough, whether or not stripped of bark or sapwood, or roughly squared, untreated, n...",M,"4,185,322.00",False,13.56
63,China,Angola,2018,44,440725,"Wood, tropical; dark red meranti, light red meranti and meranti bakau, sawn or chipped lengthwis...",M,"28,900.00",False,0.09
64,China,Angola,2018,44,440727,"Wood, tropical; sapelli, sawn or chipped lengthwise, sliced or peeled, whether or not planed, sa...",M,"191,783.00",False,0.62
